导入第三方库

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np
import math

读取数据

In [2]:
excel_path = "date_weather_type.xlsx"

with pd.ExcelFile(excel_path) as excel:
    dateGlass_pb = pd.read_excel(excel, sheet_name="pb")
    dateGlass_hk = pd.read_excel(excel, sheet_name="hk")

获取数据的平均值,简单聚合数据

In [3]:
# 还是在dataframe中操作数据
def date_mean(dateGlass):

    # 获取对应数据
    weather_time_date = dateGlass.iloc[:,3:18]
    # 按 weather_time 进行分组
    date_chemical_contene = weather_time_date.groupby(by=["weather_time"])
    # 求均值
    mean_values = date_chemical_contene.mean()
    # 更改行标签
    mean_values.index = range(1, mean_values.shape[0]+1)
    return mean_values

In [4]:
mean_values_pb = date_mean(dateGlass_pb)
mean_values_hk = date_mean(dateGlass_hk)

---

进行多项式回归的计算,单个化学成分

In [5]:
def regression_analyse(mean_values):

    # 时间点
    t = np.array([1, 2, 3, 4])
    # 化学成分
    col = mean_values.columns.shape[0]
    # 列表 存储模型和多项式转化器
    models_and_transformers = []
    # 循环求解
    for idx in range(col):
        print(f"{mean_values_hk.columns[idx]}成分的多项式回归分析:")
        # 化学成分数据
        y = mean_values_hk.iloc[:,idx]
        # 添加一次项和常数项（PolynomialFeatures需要）
        X = t[:, np.newaxis]  # 将t转换为二维数组
        # degree=2 表示二次多项式, 添加多项式特征
        poly_features = PolynomialFeatures(degree=2, include_bias=False)
        X_poly = poly_features.fit_transform(X)
        # 构建回归模型
        model = LinearRegression()
        # 拟合数据
        model.fit(X_poly, y)
        y_poly_pred = model.predict(X_poly)
        r_squared = r2_score(y, y_poly_pred)
        print(f"R^2 值: {r_squared}")
        # 获取多项式回归的系数
        coefficients = model.coef_
        intercept = model.intercept_
        # 展示多项式回归方程
        print(f"多项式回归方程: y = {intercept:.4f} + {coefficients[0]:.4f} * t + {coefficients[1]:.4f} * t^2")
        print("-"*20)

        # 将模型和多项式特征转换器作为一个元组存入列表
        models_and_transformers.append((model,r_squared, poly_features))
    return models_and_transformers
        

In [6]:
# 获取预测模型以及多项式转化器
models_and_trans_pb = regression_analyse(mean_values_pb)
models_and_trans_hk = regression_analyse(mean_values_hk)

二氧化硅(SiO2)成分的多项式回归分析:
R^2 值: 0.2835496334167078
多项式回归方程: y = 40.0770 + 27.9076 * t + -4.9765 * t^2
--------------------
氧化钠(Na2O)成分的多项式回归分析:
R^2 值: 0.3999999999999999
多项式回归方程: y = -2.1156 + 3.2439 * t + -0.7052 * t^2
--------------------
氧化钾(K2O)成分的多项式回归分析:
R^2 值: 0.013292579105604219
多项式回归方程: y = 10.6938 + -2.4005 * t + 0.4763 * t^2
--------------------
氧化钙(CaO)成分的多项式回归分析:
R^2 值: 0.027228202490758524
多项式回归方程: y = 6.9548 + -2.0534 * t + 0.4455 * t^2
--------------------
氧化镁(MgO)成分的多项式回归分析:
R^2 值: 0.9997335291258481
多项式回归方程: y = 3.7958 + -2.8222 * t + 0.5587 * t^2
--------------------
氧化铝(Al2O3)成分的多项式回归分析:
R^2 值: 0.6064487973838513
多项式回归方程: y = 14.9371 + -7.1931 * t + 1.2106 * t^2
--------------------
氧化铁(Fe2O3)成分的多项式回归分析:
R^2 值: 0.9530782012824836
多项式回归方程: y = 7.8229 + -5.4348 * t + 1.0012 * t^2
--------------------
氧化铜(CuO)成分的多项式回归分析:
R^2 值: 0.8912751068619031
多项式回归方程: y = 6.6541 + -4.0038 * t + 0.7779 * t^2
--------------------
氧化铅(PbO)成分的多项式回归分析:
R^2 值: 0.9063564954700923
多项式回归方程: y

----

预测化学成分含量

首先获取数据--> 需要预测的数据,以及对应的标准化因子

In [7]:
def predict_no_weather(dateGlass,models_and_trans):

    # 获取除了未风化的数据 : 化学成分含量, 标准化因子, weather_time, 采样点
    need_predict_chem_date = dateGlass[dateGlass["weather_time"] != 0].iloc[:,3:17]
    date_standard = dateGlass[dateGlass["weather_time"] != 0].iloc[:,2]
    date_weather_time = dateGlass[dateGlass["weather_time"] != 0].iloc[:,17]
    date_sample = dateGlass[dateGlass["weather_time"] != 0].iloc[:,1]

    date_row = need_predict_chem_date.shape[0]
    # 使用array数组进行存储
    predict_date = np.zeros((date_row, need_predict_chem_date.shape[1]))
    # 对每个数据进行预测
    for date_item_idx in range(date_row):
        for idx, model_and_tran in enumerate(models_and_trans):
            # 原数据中的化学成分含量
            npc_date = float(need_predict_chem_date.iloc[date_item_idx,idx])
            # weather_time
            npc_date_weather_time = date_weather_time.iloc[date_item_idx]
            # 标准化因子
            npc_date_standard = date_standard.iloc[date_item_idx]
            # 预测的数据
            need_predict_time = np.array([1,2,3,4])
            # 将新数据转换为二维数组
            new_need_predict_time = need_predict_time[:, np.newaxis]
            # 获取模型和多项式转化模型
            model, r_squared, tran = model_and_tran
            if r_squared >= 0.9:
                # 进行预测
                time_poly = tran.transform(new_need_predict_time)
                predict_pred = model.predict(time_poly)
                # 获取相应时间点预测数据
                weather_predict_date = float(predict_pred[npc_date_weather_time])
                # 计算未风化时间点数据
                if npc_date > weather_predict_date:
                    no_weather_date = (math.sqrt(npc_date - weather_predict_date) + predict_pred[0]) / npc_date_standard
                else:
                    no_weather_date = (math.sqrt(weather_predict_date - npc_date) + predict_pred[0]) / npc_date_standard
                # 存储数据
                predict_date[date_item_idx,idx] = no_weather_date
            else:
                predict_date[date_item_idx,idx] = npc_date / npc_date_standard
    # 保存为dataFrame数据
    nw_date_df = pd.DataFrame(predict_date,
                              columns=need_predict_chem_date.columns,
                              index=date_sample)

    return nw_date_df

In [8]:
nw_weather_hk = predict_no_weather(dateGlass_hk, models_and_trans_hk)
nw_weather_pb = predict_no_weather(dateGlass_pb, models_and_trans_pb)

---

存入excle文件中

In [9]:
with pd.ExcelWriter("predict_no_weather_date.xlsx") as writer:
    nw_weather_pb.to_excel(writer, sheet_name="pb_nw_date")
    nw_weather_hk.to_excel(writer, sheet_name="hk_nw_date")
    mean_values_pb.to_excel(writer,sheet_name="pb_mean")
    mean_values_hk.to_excel(writer,sheet_name="hk_mean")